# DEPRESSION HELPER

## Scraper PRAW Portion

In [1]:
import praw
flairs = ['flair_name:"PROVIDING SUPPORT"','flair_name:"PROVIDING ADVICE"','flair_name:"MOTIVATION"','flair_name:"INSPIRATION"']

reddit = praw.Reddit(client_id='2n-JkPtToluQ1A',
                     client_secret='2GsEl4UykIXidjZUTtnHFItoR1Y',
                     user_agent='cs410')
print(reddit.read_only)
s = [0,0,0,0]
f = open('content.txt', "wb")
for i in range(len(flairs)):
    print('Adding', flairs[i], 'to content.txt.')
    for submission in reddit.subreddit('depression_help').search(flairs[i], syntax='cloudsearch', limit=1000):
        cur = ""
        title = submission.title
        title = title.strip('\n')
        #title = title.encode("utf-8")
        content = submission.selftext.strip()
        #content = content.encode("utf-8")
        cur = title + content + '/n'
        #cur += content
        f.write(cur.encode("utf-8"))
        s[i]+=1
print('Documents per filter/flair:',s) 
f.close()
   
with open('content.txt', "r", encoding="utf8") as f:
    lines = f.readlines()
f.close()

with open('data/helpDocs.txt', "w", encoding="utf8") as f:
    for line in lines:
        if len(line) > 200:
            f.write(line)    
f.close()            
print('Stripping and filtering.')
uniqlines = set(open('data/helpDocs.txt', encoding="utf8").readlines())
bar = open('data/helpDocs.txt', 'w',encoding="utf8").writelines(set(uniqlines))

fn = open('data/helpDocs.txt', "r",encoding="utf8")
lines = fn.readlines()

avg = sum([len(line.strip('\n')) for line in lines]) / len(lines)
print("Average length of a post is", avg)
print("\nAll documents added to data/helpDocs.txt!")
    

True
Adding flair_name:"PROVIDING SUPPORT" to content.txt.
Adding flair_name:"PROVIDING ADVICE" to content.txt.
Adding flair_name:"MOTIVATION" to content.txt.
Adding flair_name:"INSPIRATION" to content.txt.
Documents per filter/flair: [78, 66, 71, 60]
Stripping and filtering.
Average length of a post is 550.6910828025477

All documents added to data/helpDocs.txt!


### PSLA Portion

In [2]:
import numpy as np
import math
import sys,os
from collections import Counter 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def normalize(input_matrix):
    row_sums = input_matrix.sum(axis=1)
    assert (np.count_nonzero(row_sums)==np.shape(row_sums)[0]) # no row should sum to zero
    new_matrix = input_matrix / row_sums[:, np.newaxis]
    return new_matrix

       
class Corpus(object):
    def __init__(self, document):
        self.documents = []
        self.vocabulary = []
        self.likelihoods = []
        self.document = document
        #self.documents_path = documents_path
        self.term_doc_matrix = None 
        self.document_topic_prob = None  # P(z | d)
        self.topic_word_prob = None  # P(w | z)
        self.topic_prob = None  # P(z | d, w)
        self.k = 0
        self.number_of_documents = 0
        self.vocabulary_size = 0

    def build_corpus(self,line):
        stop_words = set(stopwords.words('english')) 
        filtered = [word for word in line[1:].split() if not word.lower() in stop_words]
        self.documents.append(filtered)
        self.number_of_documents = 1

    def build_vocabulary(self):
        for i in range(0,self.number_of_documents):
            for j in range(0,len(self.documents[i])):
                if self.documents[i][j] not in self.vocabulary:
                    self.vocabulary.append(self.documents[i][j])
                    self.vocabulary_size+=1

    def build_term_doc_matrix(self):
        self.term_doc_matrix = np.zeros((self.number_of_documents,self.vocabulary_size))
        for i in range(0,self.number_of_documents):
            for j in range(0,self.vocabulary_size):
                docCount = Counter(self.documents[i])
                docDict = dict(docCount)
                if self.vocabulary[j] in docDict:
                    self.term_doc_matrix[i][j] = docDict[self.vocabulary[j]]
                else:
                    self.term_doc_matrix[i][j] = 0    


    def initialize_randomly(self, number_of_topics):
        self.k = number_of_topics
        self.document_topic_prob = np.random.rand(self.number_of_documents,number_of_topics)
        self.document_topic_prob = normalize(self.document_topic_prob)

        self.topic_word_prob = np.random.rand(number_of_topics,len(self.vocabulary))
        self.topic_word_prob = normalize(self.topic_word_prob)
        #pass    # REMOVE THIS
        

    def initialize_uniformly(self, number_of_topics):
        self.document_topic_prob = np.ones((self.number_of_documents, number_of_topics))
        self.document_topic_prob = normalize(self.document_topic_prob)

        self.topic_word_prob = np.ones((number_of_topics, len(self.vocabulary)))
        self.topic_word_prob = normalize(self.topic_word_prob)

    def initialize(self, number_of_topics, random=False):
        if random:
            self.initialize_randomly(number_of_topics)
        else:
            self.initialize_uniformly(number_of_topics)

    def expectation_step(self):
        for i in range(0,self.number_of_documents):
            for j in range(0,self.vocabulary_size):
                denom = 0
                for k in range(0,self.k):
                    self.topic_prob[i,k,j] = self.document_topic_prob[i,k]*self.topic_word_prob[k,j]
                    denom+=self.topic_prob[i,k,j]
                if denom==0:
                    for k in range(0,self.k):
                        self.topic_prob[i,k,j]=0
                else:
                    for k in range(0,self.k):
                        self.topic_prob[i,k,j]/=denom

    def maximization_step(self, number_of_topics):
        # update P(w | z)
        for k in range(0,number_of_topics):
            denom = 0
            for j in range(0,self.vocabulary_size):
                self.topic_word_prob[k][j] = 0
                for i in range(0,self.number_of_documents):
                    self.topic_word_prob[k,j]+=self.term_doc_matrix[i,j]*self.topic_prob[i,k,j]
                denom+=self.topic_word_prob[k,j]
            if denom ==0:
                for j in range(0,self.vocabulary_size):
                    self.topic_word_prob[k,j]=1/self.vocabulary_size
            else:
                for j in range(0,self.vocabulary_size):
                    self.topic_word_prob[k,j]/=denom
        self.topic_word_prob = normalize(self.topic_word_prob)
        # update P(z | d)
        for i in range(0,self.number_of_documents):
            for k in range(0,number_of_topics):
                self.document_topic_prob[i][k]=0
                denom = 0
                for j in range(0,self.vocabulary_size):
                    self.document_topic_prob[i][k]+=self.term_doc_matrix[i][j]*self.topic_prob[i,k,j]
                    denom +=self.term_doc_matrix[i][j]
                if denom==0:
                    self.document_topic_prob[i][k]=1/number_of_topics
                else:
                    self.document_topic_prob[i][k]/=denom
        self.document_topic_prob = normalize(self.document_topic_prob)

    def calculate_likelihood(self, number_of_topics):
        log_likelihood = 0
        for i in range(0,self.number_of_documents):
            for j in range(0,self.vocabulary_size):
                tmp=0
                for k in range(0,number_of_topics):
                    tmp+=self.topic_word_prob[k][j]*self.document_topic_prob[i][k]
                if tmp>0:
                    log_likelihood+=self.term_doc_matrix[i][j]*math.log(tmp)
        return log_likelihood

    def plsa(self, number_of_topics, max_iter, epsilon):
        self.build_term_doc_matrix()

        # P(z | d, w)
        self.topic_prob = np.zeros([self.number_of_documents, number_of_topics, self.vocabulary_size], dtype=np.float)

        # P(z | d) P(w | z)
        self.initialize(number_of_topics, random=True)

        current_likelihood = 0.0

        for iteration in range(max_iter):
            self.expectation_step()
            self.maximization_step(number_of_topics)
            current_likelihood = self.calculate_likelihood(number_of_topics)

def main():
    documents_path = 'data/userTranscript.txt'
    helpdocs_path = 'data/helpDocs.txt'

    # build help doc database, takes a little while to do... ~3min
    helpDocs = []
    with open(helpdocs_path,encoding="utf-8") as fp2:
        for line in fp2:
            corpus = Corpus(line)  # instantiate corpus
            corpus.build_corpus(line)
            corpus.build_vocabulary()
            number_of_topics = 20
            max_iterations = 50
            epsilon = 0.001
            corpus.plsa(number_of_topics, max_iterations, epsilon)

            idx = np.argmax(corpus.document_topic_prob)
            chosen_topic = corpus.topic_word_prob[idx]
            chosen_word_idx = chosen_topic.argsort()[-20:][::-1]
            help_topics = []
            for idx in chosen_word_idx:
                chosen_word = corpus.vocabulary[idx]
                help_topics.append(chosen_word)
            helpDocs.append(help_topics)
    print('Help doc db built')

    # save a local data structure for all help docs for future usage
    allHelpDocs = open('data/helpDocs.txt', "r", encoding="utf8")
    docs = allHelpDocs.read().splitlines()

    # now for each user transcript...
    with open(documents_path,encoding="utf8") as fp:

        # for each user transcript, recommend documents!
        for line in fp:
            corpus = Corpus(line)  # instantiate corpus
            corpus.build_corpus(line)
            corpus.build_vocabulary()
            number_of_topics = 20
            max_iterations = 50
            epsilon = 0.001
            corpus.plsa(number_of_topics, max_iterations, epsilon)

            print('***** TOPIC WORD DISTR *****')
            idx = np.argmax(corpus.document_topic_prob)
            chosen_topic = corpus.topic_word_prob[idx]
            chosen_word_idx = chosen_topic.argsort()[-3:][::-1]
            topics = []
            for idx in chosen_word_idx:
                chosen_word = corpus.vocabulary[idx]
                topics.append(chosen_word)
            print(topics)
            
            # compare the user transcript vector with every help doc vector, get top 10 documents!
            sims = []
            for helpDoc in helpDocs:
                a = set(helpDoc) 
                b = set(topics)
                c = a.intersection(b)

                #jacquard similarity: https://towardsdatascience.com/overview-of-text-similarity-metrics-3397c4601f50
                sim = float(len(c)) / (len(a) + len(b) - len(c))
                sims.append(sim)
            sims = np.array(sims)
            # get the 5 best doc

            fiveBestDocs = sims.argsort()[-5:][::-1]
            print('***** HERES WHAT I RECOMMEND... *****')
            for idx in fiveBestDocs:
                # print(idx)
                print(docs[idx])



if __name__ == '__main__':
    main()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Konrad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Help doc db built
***** TOPIC WORD DISTR *****
['around', 'school', 'really']
***** HERES WHAT I RECOMMEND... *****
[See trailer here](https://youtu.be/-ug1TE-kdCM)/n1000% true!/nIdk who needs to hear this today, but you're important and you deserve to be happy./nI'm happyI made a post awhile ago about my situation and wanna update that I'm dropping out of school going full time and I'll be much more happy/nI texted my dad while he was at work because I’ve been feeling really depressed today and his response made me cry. I love my dad./nkeep doing what works for you with depression (grateful for gratitude br...This is part of my 365 days of gratitude to work through my depression and the spiral into suicide attempts over the last two years. 
I just needed to get this out there and tell someone, I dont really have anyone to open up to./nLeave your mark on the world, not on yourselves.You know you are more intelligent than those around you. People like us don't just accept what's handed 

***** TOPIC WORD DISTR *****
['dont', 'need', 'fucking']
***** HERES WHAT I RECOMMEND... *****
It might be for different reasons. We might come from different angles. But fuck, people like me.. we fucking love you, we support you, and we want the fucking best for you. *We don't want to lose you.* We've lost so, so much. Please stay here, for us. Because we need you here. We need you to need us. I know that sometimes it can seem so easy to give up.. trust me, as a person who has been here as a survivor on this end, *I get it*. 
* you don't have to be good at art, or have good handwriting (look at mine, it's terrible), you just have to try! It really doesn't need to look perfect, or like it was ripped from pintrest or IG, it just needs to be functional for you.
I'm not looking for sympathy here. I don't need it. All I'm asking is that you stay here for me, so I can stay here for you. Being able to help you helps me. If you stay for no else and nothing else.. please.. stay for me. Because

***** TOPIC WORD DISTR *****
['someone', 'like', 'worse']
***** HERES WHAT I RECOMMEND... *****
&#x200B;/nI made a video talking about what depression really feels like for those of you that don't completely understand.For those of you that would like to know a little bit more about what depression feels like I hope this helps. 
* you don't have to be good at art, or have good handwriting (look at mine, it's terrible), you just have to try! It really doesn't need to look perfect, or like it was ripped from pintrest or IG, it just needs to be functional for you.
Its like a cat chasing its own tail trying to stop it,when all effort is gone then the movement stops! Then the cat relaxes,then it realizes it was depression itself all along and chased its own tail trying to do something about it,
* I know doing this everyday seems a bit of a drag, but I find the practice of making these boxes really soothing and meditative. It's a practice. It's like I'm investing in myself, the more work I d

***** TOPIC WORD DISTR *****
['ago.', 'like', 'ran']
***** HERES WHAT I RECOMMEND... *****
&#x200B;/nI made a video talking about what depression really feels like for those of you that don't completely understand.For those of you that would like to know a little bit more about what depression feels like I hope this helps. 
Ever since, I’ve been experiencing the worst depression I’ve had in forever. The rollercoaster I’ve been riding for the last week brought me to the point of wanting to just end it all a few nights ago. 
* you don't have to be good at art, or have good handwriting (look at mine, it's terrible), you just have to try! It really doesn't need to look perfect, or like it was ripped from pintrest or IG, it just needs to be functional for you.
Its like a cat chasing its own tail trying to stop it,when all effort is gone then the movement stops! Then the cat relaxes,then it realizes it was depression itself all along and chased its own tail trying to do something about it,
*

...

***** TOPIC WORD DISTR *****
[',', 'body', 'Depression']
***** HERES WHAT I RECOMMEND... *****
Back when I suffered from depression, thinking like this and exercise helped me a lot, so now I ask you, what made you happy or what are you grateful for today?/n5 Natural Remedies for Depression That I've Tried | Depression Medication/nSupport groupchat?I’ve been thinking now and I was wondering if any if you want to create a support group chat maybe on snapchat or something. Just so when we need to tell people something or need advice , or even just chat we can have that chat. Wherever it will take place, I will not name the chat something like support group because I know some of you, as well as me, does not want friends or family finding it. If you’re interested just comment or pm me./nHappy Valentines Day for all of those in struggle or are lonely.Even though I have never had a girlfriend (mostly because how fucking weird/strange I am), I wanted to share the love this year. 
My Depressio

***** TOPIC WORD DISTR *****
["I'm", 'know', 'going']
***** HERES WHAT I RECOMMEND... *****
* I know doing this everyday seems a bit of a drag, but I find the practice of making these boxes really soothing and meditative. It's a practice. It's like I'm investing in myself, the more work I do.
Flash forward, I started college. I landed myself in a large city where I knew absolutely anyone. I know how to speak to people, but I have too much social anxiety to willingly put myself out there and make friends, so I had no place that I really belonged socially. On top of it all, I was taking 17 credits, I had no life outside of my coursework, and my now long-distance boyfriend had dumped me. All I could do was cry myself to sleep every single night at that point, and that’s when I remember everything going downhill. I thought I was at my worst, but little did I know that was just the start.
Classmates will always be competitive and a lot of people may see my grades as bad but I think it's imp

***** TOPIC WORD DISTR *****
['heal', 'could', '“fix”']
***** HERES WHAT I RECOMMEND... *****
I don’t talk about my life that much because I’m a private person. Plus alot of my friends and alot of the other girls at Eva Carlston had situations that were WAY worse than mine. I had my grandparents, who cared enough to make me do what needed to be done (after the horse s\*\*t place anyway). I got pretty lucky at Eva Carlston too. I know some kids have bad experiences in residential treatment centers but, FWIW, I got out of it way more than what I put in. Some of that could have been the location and not being stuck out in bumble but for me it was probably better than getting put on meds or going to jail or whatever else I was heading for.
It is crucial to write down your automatic thoughts and rational responses; do not try to do the exercise in your head. Writing them down forces you to develop much more objectivity than you could ever achieve by letting responses swirl through your mind

***** TOPIC WORD DISTR *****
['sewer', 'house', 'fix']
***** HERES WHAT I RECOMMEND... *****
Even when i look at my life, my parents went through an awful divorce, i had to move out the house for my studies, and move to a new country alone, with no friends, it was quite tough on my psyche, and i know i have it easy.
So we are learning to strengthen ourselves on strong foundations, instead of foundations like feelings, moods, relationships, other people's opinions, and things that are constantly shifting from day to day, moment to moment. There's nothing wrong with any of those things by themselves or in their proper context. But in the context of building core foundations for a healthier inner-person, none of those things stay still long enough to keep a house from falling over.
1. Personification: I am being very intentional about externalizing the issue. It's usually the opposite that's taught. Internalize it, "accept that it's a part of who you are and always will be." If it's a par

***** TOPIC WORD DISTR *****
['it.', 'I’ve', 'children']
***** HERES WHAT I RECOMMEND... *****
One important point I haven't mentioned yet is I’ve learned to be very intentional about making these choices and perspectives a foundation outside of the storm. So if the storm does come I’m already prepared to deal with it.
Ever since, I’ve been experiencing the worst depression I’ve had in forever. The rollercoaster I’ve been riding for the last week brought me to the point of wanting to just end it all a few nights ago. 
I'm not looking for sympathy here. I don't need it. All I'm asking is that you stay here for me, so I can stay here for you. Being able to help you helps me. If you stay for no else and nothing else.. please.. stay for me. Because I need you to need me.
https://paindoctor.com/free-therapy-online//nWhat do you need help with most?Hi guys, I am a recovered depressed person. I went through severe depression and anxiety for my whole life up until my recovery over the past few